In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [2]:
import numpy as np
import os



valve_ids = np.arange(12,19+1)
hyb_ids = np.arange(12,19+1)
reg_names = [f'M{_i}_bit{_i*2-1}-{_i*2}' for _i in hyb_ids]
print(reg_names)

['M12_bit23-24', 'M13_bit25-26', 'M14_bit27-28', 'M15_bit29-30', 'M16_bit31-32', 'M17_bit33-34', 'M18_bit35-36', 'M19_bit37-38']


In [3]:
source_folder = r'D:\Pu\20210717-P_brain_MO4_MERFISH'
target_drive = r'\\KOLMOGOROV\Chromatin_NAS_5'
imaging_protocol = r'Conv_Zscan_750_647_561_s13_n1000_5Hz'

In [4]:
cmd_seq = Element('command_sequence')

for _vid, _hid, _rname in zip(valve_ids, hyb_ids, reg_names):
    # comments
    comment = Comment(f"Hyb {_hid} for {_rname}")
    cmd_seq.append(comment)
    # TCEP
    tcep = SubElement(cmd_seq, 'valve_protocol')
    tcep.text = "MERFISH TCEP"
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'delay')
    adt_incubation.text = "600000"
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "MERFISH Wash"
    # readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow STORM Buffer"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'delay')
    adt_incubation.text = "60000"
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    
    # hybridization loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=imaging_protocol)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "5000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)


In [5]:
print( prettify(cmd_seq))

<?xml version="1.0" ?>
<command_sequence>
	<!--Hyb 12 for M12_bit23-24-->
	<valve_protocol>MERFISH TCEP</valve_protocol>
	<valve_protocol>Hybridize 12</valve_protocol>
	<delay>600000</delay>
	<valve_protocol>MERFISH Wash</valve_protocol>
	<valve_protocol>Flow STORM Buffer</valve_protocol>
	<delay>60000</delay>
	<change_directory>D:\Pu\20210717-P_brain_MO4_MERFISH\H12M12_BIT23-24</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Conv_Zscan_750_647_561_s13_n1000_5Hz"> </item>
	</loop>
	<delay>5000</delay>
	<copy_directory>
		<source_path>D:\Pu\20210717-P_brain_MO4_MERFISH\H12M12_BIT23-24</source_path>
		<target_path>\\KOLMOGOROV\Chromatin_NAS_5\20210717-P_brain_MO4_MERFISH\H12M12_BIT23-24</target_path>
		<delete_source>True</delete_source>
	</copy_directory>
	<!--Hyb 13 for M13_bit25-26-->
	<valve_protocol>MERFISH TCEP</valve_protocol>
	<valve_protocol>Hybridize 13</valve_protocol>
	<delay>600000</delay>
	<valve_protocol>MERFISH W